In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os, random
import cv2
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam 
from tensorflow import keras
import keras.layers as layers


# animal categories
categories = ['mountain','ocean','street','forest']
no_cate=len(categories)
no_cate #number of categories


4

In [ ]:
# load data，
path ='/content/drive/MyDrive/project/train_test_val_XY.npz'
train_test_val_XY = np.load(path)

# extract data
trainX = train_test_val_XY['trainX']
trainY = train_test_val_XY['trainY']

testX = train_test_val_XY['testX']
testY = train_test_val_XY['testY']

validationX = train_test_val_XY['validationX']
validationY = train_test_val_XY['validationY']

print(trainX.shape)
print(validationX.shape)
print(testX.shape)
print(trainY.shape)
print(validationY.shape)
print(testY.shape)

(5600, 224, 224, 3)
(1200, 224, 224, 3)
(1200, 224, 224, 3)
(5600, 4)
(1200, 4)
(1200, 4)


# **Model Fitting 1: AlexNet**

In [ ]:
HEIGHT =224
WIDTH =224
N_CHANNELS = 3

np.random.seed(1000)

#Instantiation
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Convolution2D(filters=96, input_shape=(HEIGHT,WIDTH,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Convolution2D(filters=256, kernel_size=(5,5), strides=1, padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2),strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Convolution2D(filters=384, kernel_size=(3,3), strides=1, padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Convolution2D(filters=384, kernel_size=(3,3), strides=1, padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Convolution2D(filters=256, kernel_size=(3,3), strides=1, padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=1, padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.2))

#Output Layer
AlexNet.add(Dense(no_cate))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

In [ ]:
# Define learning rate
# lr_candidate = [0.0001, 0.001, 0.01, 0.1]

# Define epochs
epochs = 200

# Define learning rate
lr = 0.1

# Define optimzer w.r.t. different learning rate
opt = Adam(learning_rate=lr)

# Initialize tensorboard
logdir = os.path.join("/content/drive/MyDrive/project/logs", "lr = " + str(lr))
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Model Compile
AlexNet.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Model fitting (training)
AlexNet.fit(trainX,trainY,
            batch_size=128, 
            epochs=epochs,
            validation_data=(validationX,validationY),
            callbacks=[tensorboard_callback])

In [ ]:
# %load_ext tensorboard

In [ ]:
# %tensorboard --logdir logs --port=6009

In [ ]:
# AlexNet.save("/content/drive/MyDrive/project/alexnet" + str(lr) + ".h5")

In [ ]:
AlexNet.evaluate(testX,testY)

38/38 [==============================] - 1s 19ms/step - loss: 1.4772 - accuracy: 0.7517


[1.4772168397903442, 0.7516666650772095]

# **Model Fitting 2: ResNet34**

In [ ]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,padding="SAME", use_bias=False)
class ResidualUnit(keras.layers.Layer):
  def __init__(self, filters, strides=1, activation="relu", **kwargs):
    super().__init__(**kwargs)
    self.activation = keras.activations.get(activation)
    self.main_layers = [
                        DefaultConv2D(filters, strides=strides),
                        keras.layers.BatchNormalization(),
                        self.activation,
                        DefaultConv2D(filters),
                        keras.layers.BatchNormalization()
    ]
    self.skip_layers = []
    if strides > 1:
      self.skip_layers = [
                          DefaultConv2D(filters, kernel_size=1, strides=strides),
                          keras.layers.BatchNormalization()
      ]

def call(self, inputs):
  Z = inputs
  for layer in self.main_layers:
    Z = layer(Z)
  skip_Z = inputs
  for layer in self.skip_layers:
    skip_Z = layer(skip_Z)
  return self.activation(Z + skip_Z)

In [ ]:
resnet34 = keras.models.Sequential()
resnet34.add(DefaultConv2D(64, kernel_size=7, strides=2,input_shape=[224,224, 3]))
resnet34.add(keras.layers.BatchNormalization())
resnet34.add(keras.layers.Activation("relu"))
resnet34.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))

prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
  strides = 1 if filters == prev_filters else 2
  resnet34.add(ResidualUnit(filters, strides=strides))
  prev_filters = filters

resnet34.add(keras.layers.GlobalAvgPool2D())
resnet34.add(keras.layers.Flatten())
resnet34.add(keras.layers.Dense(4, activation="softmax"))

In [ ]:
# Define learning rate
# lr_candidate = [0.0001, 0.001]

# Define epochs
epochs = 200

# Define learning rate
lr = 0.0001

# Define optimzer w.r.t. different learning rate
opt = Adam(learning_rate=lr)

# Initialize tensorboard
logdir = os.path.join("/content/drive/MyDrive/project/logs", "resnet34", "lr = " + str(lr))
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# logdir = os.path.join("logs", "vgg", "lr = " + str(lr))
# tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Model Compile
resnet34.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Model fitting (training)
resnet34.fit(trainX,trainY,
            batch_size=64,
            epochs=epochs,
            validation_data=(validationX,validationY),
            callbacks=[tensorboard_callback])


# It can be used to reconstruct the model identically.
# reconstructed_model = keras.models.load_model("model.h5")

In [ ]:
resnet34.evaluate(testX,testY)

38/38 [==============================] - 0s 12ms/step - loss: 0.7422 - accuracy: 0.7192


[0.7422178387641907, 0.7191666960716248]

In [ ]:
# resnet34.save_weights("/content/drive/Shareddrives/zhufang147/resnet34" + str(lr) + ".ckpt")


# **Model Fitting 3: ResNet50**

In [ ]:
class ResidualUnit(keras.layers.Layer):
  def __init__(self, filters, strides=1, activation="relu", **kwargs):
    super().__init__(**kwargs)
    self.activation = keras.activations.get(activation)
    self.main_layers = [
                        DefaultConv2D(filters, kernel_size=1,strides=1),
                        keras.layers.BatchNormalization(),
                        self.activation,
                        DefaultConv2D(filters, kernel_size=3,strides=strides),
                        keras.layers.BatchNormalization(),
                        self.activation,
                        DefaultConv2D(filters*4,kernel_size=1,strides=1),
                        keras.layers.BatchNormalization()
    ]
    self.skip_layers = []
    if strides > 1:
      self.skip_layers = [
                          DefaultConv2D(filters*4, kernel_size=1, strides=strides),
                          keras.layers.BatchNormalization()
      ]

def call(self, inputs):
  Z = inputs
  for layer in self.main_layers:
    Z = layer(Z)
  skip_Z = inputs
  for layer in self.skip_layers:
    skip_Z = layer(skip_Z)
  return self.activation(Z + skip_Z)

In [ ]:
resnet50 = keras.models.Sequential()
resnet50.add(DefaultConv2D(64, kernel_size=7, strides=2,input_shape=[224, 224, 3]))
resnet50.add(keras.layers.BatchNormalization())
resnet50.add(keras.layers.Activation("relu"))
resnet50.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))

prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
  strides = 1 if filters == prev_filters else 2
  resnet50.add(ResidualUnit(filters, strides=strides))
  prev_filters = filters

resnet50.add(keras.layers.GlobalAvgPool2D())
resnet50.add(keras.layers.Flatten())
resnet50.add(keras.layers.Dense(4, activation="softmax"))

In [ ]:
# Define learning rate
# lr_candidate = [0.0001, 0.001]

# Define epochs
epochs = 200

# Define learning rate
lr = 0.1

# Define optimzer w.r.t. different learning rate
opt = Adam(learning_rate=lr)

# Initialize tensorboard
logdir = os.path.join("/content/drive/MyDrive/project/logs", "resnet50", "lr = " + str(lr))
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# logdir = os.path.join("logs", "vgg", "lr = " + str(lr))
# tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Model Compile
resnet50.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Model fitting (training)
resnet50.fit(trainX,trainY,
            batch_size=64,
            epochs=epochs,
            validation_data=(validationX,validationY),
            callbacks=[tensorboard_callback])


# It can be used to reconstruct the model identically.
# reconstructed_model = keras.models.load_model("model.h5")

In [ ]:
resnet50.evaluate(testX,testY)

38/38 [==============================] - 0s 11ms/step - loss: 0.8486 - accuracy: 0.6617


[0.8485937714576721, 0.6616666913032532]

In [ ]:
!nvidia-smi

Fri May 13 15:41:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    26W /  70W |  14586MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------